In [ ]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
# Set up constants
TIMEFRAMES = [7, 14, 30]
PREDICTIONS = [1, 5]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

# Load labels
labels = pd.read_csv('labels/labels.csv')

In [ ]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 7:
            # Crop the image using the bounding rectangle
            crop = img[105:105+115, 80:80+38]
        elif number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [ ]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

In [ ]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [ ]:
# Sort the DataFrame by date and train on MSCIWorld only
labels = labels[labels['Image'].str.contains('MSCIWorld')]
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

In [ ]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

In [ ]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    # Determine the number of blocks based on the input image size
    if num_blocks == 2:
        dilation_rate = (1, 1)
    elif num_blocks == 3:
        dilation_rate = (1, 2)
    else:
        dilation_rate = (1, 3)

    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Add Convolutional layer
        if i == 0:  # Apply specific strides and dilation rate only for the first layer
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 3), padding='same', 
                             input_shape=input_shape if i == 0 else None))
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 1), dilation_rate=dilation_rate, padding='same', 
                             input_shape=input_shape if i == 0 else None))
        else:
            model.add(Conv2D(filters, kernel_size=(5, 3), padding='same'))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer
        model.add(MaxPooling2D(pool_size=(2, 1)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

    return model

In [ ]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i][0] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_investment += 100
    
    if long_investment > 0:
        # Calculate average long RoR
        long_avg_ror = np.mean(long_ror)
    else:
        long_avg_ror = 0
    
    if short_investment > 0:
        # Calculate average short RoR
        short_avg_ror = np.mean(short_ror)
    else:
        short_avg_ror = 0

    return long_avg_ror, long_investment, short_avg_ror, short_investment

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Short_Average_RoR', 'Short_Investment'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Initialize variables for weighted sums and total count
                weighted_accuracy_sum = 0
                weighted_precision_sum = 0
                weighted_recall_sum = 0
                weighted_f1_score_sum = 0
                weighted_hit_rate_sum = 0
                total_predictions = 0
                long_avg_ror = []
                long_investment = []
                short_avg_ror = []
                short_investment = []
                total_investment = []
                total_returns = []
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 7:
                    model = create_cnn_model(2, input_shape=(115, 38, 1))
                elif timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(2, input_shape=(120, 132, 1))
                
                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Define the rolling window size for training within the initial training set
                window_size = len(X) / 10

                # Early stopping callback
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                for window in range(5):
                    start = 0
                    if window == 0:
                        end = int(len(X) * 0.4)
                    else:
                        end = end + int(window_size)

                    # Buffer = window size
                    # Define end of buffer
                    buffer_end = end + int(window_size)

                    # Ensure buffer_end does not exceed the length of the initial training set
                    buffer_end = min(buffer_end, len(X))

                    # Split the data into training and test sets in temporal order
                    X_train_temp = X[start:end]
                    y_train_temp = y[start:end]
                    lastPrice_train_temp = lastPrice[start:end]
                    futurePrice_train_temp = futurePrice[start:end]

                    X_test_temp = X[buffer_end:] if window == 4 else X[buffer_end:buffer_end+int(window_size*2)]
                    y_test_temp = y[buffer_end:] if window == 4 else y[buffer_end:buffer_end+int(window_size*2)]
                    lastPrice_test_temp = lastPrice[buffer_end:] if window == 4 else lastPrice[buffer_end:buffer_end+int(window_size*2)]
                    futurePrice_test_temp = futurePrice[buffer_end:] if window == 4 else futurePrice[buffer_end:buffer_end+int(window_size*2)]

                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2)  
                
                    # Evaluate the model
                    y_pred = model.predict(X_test_temp)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                    num_predictions = len(y_test_temp)
                    
                    # Calculate the performance metrics for the current epoch
                    current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                    current_precision = precision_score(y_test_temp, y_pred_binary)
                    current_recall = recall_score(y_test_temp, y_pred_binary)
                    current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                    y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                    correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                    current_hit_rate = (correct_predictions / num_predictions)

                    # Update the weighted sums and total count
                    weighted_accuracy_sum += current_accuracy * num_predictions
                    weighted_precision_sum += current_precision * num_predictions
                    weighted_recall_sum += current_recall * num_predictions
                    weighted_f1_score_sum += current_f1_score * num_predictions
                    weighted_hit_rate_sum += current_hit_rate * num_predictions
                    total_predictions += num_predictions

                    long_avg_ror_temp, long_investment_temp, short_avg_ror_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                    # Total investment
                    long_avg_ror.append(long_avg_ror_temp)
                    long_investment.append(long_investment_temp)
                    short_avg_ror.append(short_avg_ror_temp)
                    short_investment.append(short_investment_temp)
                
                # replace nan values with 0
                long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
                long_investment = [0 if np.isnan(x) else x for x in long_investment]
                short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
                short_investment = [0 if np.isnan(x) else x for x in short_investment]

                # Calculate the weighted averages after the epoch loop
                weighted_average_accuracy = weighted_accuracy_sum / total_predictions
                weighted_average_precision = weighted_precision_sum / total_predictions
                weighted_average_recall = weighted_recall_sum / total_predictions
                weighted_average_f1_score = weighted_f1_score_sum / total_predictions
                weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                            
                print("Evaluation Metrics:")
                print(f"Accuracy: {weighted_average_accuracy}")
                print(f"Precision: {weighted_average_precision}")
                print(f"Recall: {weighted_average_recall}")
                print(f"F1 Score: {weighted_average_f1_score}")
                print(f"Hit Rate: {weighted_average_hit_rate}")
                print(f"Average Long RoR: {np.mean(long_avg_ror)}")
                print(f"Long Investment: {np.mean(long_investment)}")
                print(f"Average Short RoR: {np.mean(short_avg_ror)}")
                print(f"Short Investment: {np.mean(short_investment)}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': weighted_average_accuracy,
                    'Precision': weighted_average_precision,
                    'Recall': weighted_average_recall,
                    'F1_Score': weighted_average_f1_score,
                    'Hit_Rate': weighted_average_hit_rate,
                    'Long_Average_RoR': np.mean(long_avg_ror),
                    'Long_Investment': np.mean(long_investment),
                    'Short_Average_RoR': np.mean(short_avg_ror),
                    'Short_Investment': np.mean(short_investment)
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/MSCIWorld_{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as MSCIWorld_{img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/separate/MSCIWorld_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation/MSCIWorld_separate/evaluation_scores.csv'.")

In [ ]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Short_Average_RoR', 'Short_Investment'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Initialize variables for weighted sums and total count
            weighted_accuracy_sum = 0
            weighted_precision_sum = 0
            weighted_recall_sum = 0
            weighted_f1_score_sum = 0
            weighted_hit_rate_sum = 0
            total_predictions = 0
            long_avg_ror = []
            long_investment = []
            short_avg_ror = []
            short_investment = []
            total_investment = []
            total_returns = []
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 7:
                model = create_cnn_model(2, input_shape=(115, 38, 1))
            elif timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(2, input_shape=(120, 132, 1))
            #elif timeframe == 90:
                #model = create_cnn_model(3, input_shape=(120, 226, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values

            # Define the rolling window size for training within the initial training set
            window_size = len(X) / 20

            # Early stopping callback
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            for window in range(10):
                start = 0
                if window == 0:
                    end = int(len(X) * 0.4)
                else:
                    end = end + int(window_size)

                # Buffer = window size
                # Define end of buffer
                buffer_end = end + int(window_size)

                # Ensure buffer_end does not exceed the length of the initial training set
                buffer_end = min(buffer_end, len(X))

                # Split the data into training and test sets in temporal order
                X_train_temp = X[start:end]
                y_train_temp = y[start:end]
                lastPrice_train_temp = lastPrice[start:end]
                futurePrice_train_temp = futurePrice[start:end]

                X_test_temp = X[buffer_end:] if window == 4 else X[buffer_end:buffer_end+int(window_size*2)]
                y_test_temp = y[buffer_end:] if window == 4 else y[buffer_end:buffer_end+int(window_size*2)]
                lastPrice_test_temp = lastPrice[buffer_end:] if window == 4 else lastPrice[buffer_end:buffer_end+int(window_size*2)]
                futurePrice_test_temp = futurePrice[buffer_end:] if window == 4 else futurePrice[buffer_end:buffer_end+int(window_size*2)]

                #model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2, class_weight=class_weight_dict) 
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2)  
            
                # Evaluate the model
                y_pred = model.predict(X_test_temp)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                num_predictions = len(y_test_temp)
                
                # Calculate the performance metrics for the current epoch
                current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                current_precision = precision_score(y_test_temp, y_pred_binary)
                current_recall = recall_score(y_test_temp, y_pred_binary)
                current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                current_hit_rate = (correct_predictions / num_predictions)

                # Update the weighted sums and total count
                weighted_accuracy_sum += current_accuracy * num_predictions
                weighted_precision_sum += current_precision * num_predictions
                weighted_recall_sum += current_recall * num_predictions
                weighted_f1_score_sum += current_f1_score * num_predictions
                weighted_hit_rate_sum += current_hit_rate * num_predictions
                total_predictions += num_predictions

                long_avg_ror_temp, long_investment_temp, short_avg_ror_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                # Total investment
                long_avg_ror.append(long_avg_ror_temp)
                long_investment.append(long_investment_temp)
                short_avg_ror.append(short_avg_ror_temp)
                short_investment.append(short_investment_temp)
            
            # replace nan values with 0
            long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
            long_investment = [0 if np.isnan(x) else x for x in long_investment]
            short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
            short_investment = [0 if np.isnan(x) else x for x in short_investment]

            # Calculate the weighted averages after the epoch loop
            weighted_average_accuracy = weighted_accuracy_sum / total_predictions
            weighted_average_precision = weighted_precision_sum / total_predictions
            weighted_average_recall = weighted_recall_sum / total_predictions
            weighted_average_f1_score = weighted_f1_score_sum / total_predictions
            weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                        
            print("Evaluation Metrics:")
            print(f"Accuracy: {weighted_average_accuracy}")
            print(f"Precision: {weighted_average_precision}")
            print(f"Recall: {weighted_average_recall}")
            print(f"F1 Score: {weighted_average_f1_score}")
            print(f"Hit Rate: {weighted_average_hit_rate}")
            print(f"Average Long RoR: {np.mean(long_avg_ror)}")
            print(f"Long Investment: {np.mean(long_investment)}")
            print(f"Average Short RoR: {np.mean(short_avg_ror)}")
            print(f"Short Investment: {np.mean(short_investment)}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': weighted_average_accuracy,
                'Precision': weighted_average_precision,
                'Recall': weighted_average_recall,
                'F1_Score': weighted_average_f1_score,
                'Hit_Rate': weighted_average_hit_rate,
                'Long_Average_RoR': np.mean(long_avg_ror),
                'Long_Investment': np.mean(long_investment),
                'Short_Average_RoR': np.mean(short_avg_ror),
                'Short_Investment': np.mean(short_investment),
            }, ignore_index=True)

            # Save the model
            model.save(f"models/MSCIWorld_combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as MSCIWorld_combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/combined/MSCIWorld_rolling_evaluation_scores_combined.csv', index=False)
print("Evaluation scores saved to 'evaluation/combined/MSCIWorld_rolling_model_evaluation_scores.csv'.")